In [1]:
import pandas as pd
import numpy as np
import os

file_locations = {
    'tpch': ['customer/orders_customer_optimal_rep.csv', 'orders/customer_orders_optimal_rep.csv', 
             'orders/lineitem_orders_optimal_rep.csv', 'part/lineitem_part_optimal_rep.csv', 
             'part/partsupp_part_optimal_rep.csv', 'partsupp/lineitem_partsupp_optimal_rep.csv', 
             'partsupp/part_partsupp_optimal_rep.csv', 'partsupp/supplier_partsupp_optimal_rep.csv', 
             'supplier/lineitem_supplier_optimal_rep.csv', 'supplier/partsupp_supplier_optimal_rep.csv'],
    'imdb': ['cast_info/title_cast_info_optimal_rep.csv',
             'movie_companies/title_movie_companies_optimal_rep.csv', 'movie_info/title_movie_info_optimal_rep.csv', 
             'movie_info_idx/title_movie_info_idx_optimal_rep.csv', 'movie_keyword/title_movie_keyword_optimal_rep.csv',
             'title/cast_info_title_optimal_rep.csv', 'title/movie_companies_title_optimal_rep.csv',
             'title/movie_info_idx_title_optimal_rep.csv', 'title/movie_info_title_optimal_rep.csv',
             'title/movie_keyword_title_optimal_rep.csv'
            ],
    'ssb': ['customer/lineorder_customer_optimal_rep.csv', 'ddate/lineorder_ddate_optimal_rep.csv', 
           'part/lineorder_part_optimal_rep.csv', 'supplier/lineorder_supplier_optimal_rep.csv'
           ]
}


datasets = ['ssb', 'tpch', 'imdb']



dfs = []


for d in datasets:
    for f in file_locations[d]:
        dfs.append(pd.read_csv(os.path.join("../data/", d, f)))

ds = pd.concat(dfs)
ds

ds['left+right'] = ds['left_cardinality'] + ds['base_cardinality']
ds['left*right'] = ds['left_cardinality'] * ds['base_cardinality']
ds['left/right'] = ds['left_cardinality'] / ds['base_cardinality']
ds['left-right'] = ds['left_cardinality'] - ds['base_cardinality']
ds['left^2'] = ds['left_cardinality'] * ds['left_cardinality']
ds['right^2'] = ds['base_cardinality'] * ds['base_cardinality']
ds['left*logleft'] = ds['left_cardinality'] * np.log(ds['left_cardinality'])
ds['right*logright'] = ds['base_cardinality'] * np.log(ds['base_cardinality'])


In [2]:
from pygam import LinearGAM, s, f
from pygam.datasets import wage
from interpret.glassbox import ExplainableBoostingClassifier, ExplainableBoostingRegressor

m_regression_model = ExplainableBoostingRegressor
m_classification_model = ExplainableBoostingClassifier

# Classification Task

In [3]:
all_features = ['left_cardinality', 'base_cardinality',
       'selectivity_on_indexed_attr', 'left_ordered', 'base_ordered',
       'result_size', 'sel_on_indexed_attr_with_join_predicate']

key_features = ['left_cardinality', 'base_cardinality',
       'result_size']

augmented_features = all_features + \
    ['left+right', 'left*right', 'left/right',
        'left-right', 'left-right', 'left^2', 'right^2', 'left*logleft', 'right*logright']

features = augmented_features

regression_targets = ['hj_idx_cost', 'hj_seq_cost', 'nl_idx_cost', 'nl_seq_cost', 'mj_idx_cost', 'mj_seq_cost']
classification_target = ['optimal_decision']


X = ds[features]
y = ds['optimal_decision']

from sklearn.model_selection import train_test_split
from sklearn import preprocessing


X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,random_state=1)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

# scaler = preprocessing.StandardScaler().fit(X_test)
X_test = scaler.transform(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.


In [4]:
from sklearn.ensemble import RandomForestClassifier
clf = m_classification_model().fit(X_train, y_train)

acc = np.sum(clf.predict(X_test) == y_test) / len(y_test)
print(f"Test accuracy: %.2f%%" % (acc * 100))

/usr/local/lib/python3.6/dist-packages/interpret/glassbox/ebm/ebm.py:801: UserWarning: Multiclass is still experimental. Subject to change per release.
  warn("Multiclass is still experimental. Subject to change per release.")


Test accuracy: 96.12%


In [5]:
# import shap

# x_ds_train = pd.DataFrame(data=X_train[0:500], columns=features)

# shap_values = shap.TreeExplainer(clf).shap_values(x_ds_train)
# shap.summary_plot(shap_values, x_ds_train, plot_type="bar")

# Regression Task

In [6]:
# Collect all the regressors
regressors = {}

features = augmented_features

regression_targets = ['hj_idx_cost', 'hj_seq_cost', 'nl_idx_cost', 'nl_seq_cost','mj_idx_cost', 'mj_seq_cost']
classification_target = ['optimal_decision']


X = ds[features].to_numpy()
y = ds[regression_targets + classification_target]

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# quantile_transformer = preprocessing.QuantileTransformer(random_state=0).fit(X_train)
# X_train = quantile_transformer.transform(X_train)
# X_test = quantile_transformer.transform(X_test)


scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

X_test = scaler.transform(X_test)

### Hash join + index scan

In [7]:
c_y_train = np.log(y_train['hj_idx_cost'].to_numpy().reshape(-1, 1))
c_y_test = y_test['hj_idx_cost'].to_numpy().reshape(-1, 1)

# y_scaler = preprocessing.MinMaxScaler()
# c_y_train = y_scaler.fit_transform(c_y_train)

rgr = m_regression_model().fit(X_train, c_y_train)
regressors['hj_idx_cost'] = rgr
loss_in_percentage = np.average(np.abs(np.exp(rgr.predict(X_test).reshape(-1,1)) - c_y_test) / c_y_test)
print(f"Loss in percentage: +- {loss_in_percentage * 100}%")

Loss in percentage: +- 18.71261786236404%


In [8]:
# for name, importance in zip(features, rgr.feature_importances_):
#     print(name, "=", importance)

# import matplotlib.pyplot as plt

# importances = rgr.feature_importances_
# indices = np.argsort(importances)

# plt.title('Feature importances for hash join + index scan')
# plt.barh(range(len(indices)), importances[indices], color='b', align='center')
# plt.yticks(range(len(indices)), [features[i] for i in indices])
# plt.xlabel('Relative Importance')
# plt.show()

# c_y_train = y_train['hj_idx_cost'].to_numpy().reshape(-1, 1)
# c_y_test = y_test['hj_idx_cost'].to_numpy().reshape(-1, 1)

# y_scaler = preprocessing.QuantileTransformer()
# c_y_train = y_scaler.fit_transform(c_y_train)

# rgr = m_regression_model().fit(X_train, c_y_train)
# regressors['hj_idx_cost'] = rgr
# loss_in_percentage = np.average(np.abs(y_scaler.inverse_transform(rgr.predict(X_test).reshape(-1,1)) - c_y_test) / c_y_test)
# print(f"Loss in percentage: +- {loss_in_percentage * 100}%")

### Hash join + seq scan

In [9]:
c_y_train = np.log(y_train['hj_seq_cost'].to_numpy().reshape(-1, 1))
c_y_test = y_test['hj_seq_cost'].to_numpy().reshape(-1, 1)
rgr = m_regression_model().fit(X_train, c_y_train)
regressors['hj_seq_cost'] = rgr
loss_in_percentage = np.average(np.abs(np.exp(rgr.predict(X_test).reshape(-1,1)) - c_y_test) / c_y_test)
print(f"Loss in percentage: +- {loss_in_percentage * 100}%")

Loss in percentage: +- 11.229488212949125%


In [10]:
# for name, importance in zip(features, rgr.feature_importances_):
#     print(name, "=", importance)

# import matplotlib.pyplot as plt

# importances = rgr.feature_importances_
# indices = np.argsort(importances)

# plt.title('Feature importances for hash join + index scan')
# plt.barh(range(len(indices)), importances[indices], color='b', align='center')
# plt.yticks(range(len(indices)), [features[i] for i in indices])
# plt.xlabel('Relative Importance')
# plt.show()
rgr.predict(X_test).reshape(-1,1)

array([[11.09884062],
       [11.73346126],
       [11.6535318 ],
       ...,
       [12.11420642],
       [11.7524864 ],
       [ 9.95634816]])

### Nested loop + idx scan

In [11]:
c_y_train, c_y_test = np.log(y_train['nl_idx_cost'].to_numpy().reshape(-1, 1)), y_test['nl_idx_cost'].to_numpy().reshape(-1, 1)
rgr = m_regression_model().fit(X_train, c_y_train)
regressors['nl_idx_cost'] = rgr
loss_in_percentage = np.average( np.abs( np.exp(rgr.predict(X_test).reshape(-1,1)) - c_y_test) / c_y_test)
print(f"Loss in percentage: +- {loss_in_percentage * 100}%")

Loss in percentage: +- 22.567895832729576%


### nested loop + seq scan

In [12]:
c_y_train = np.log(y_train['nl_seq_cost'].to_numpy().reshape(-1, 1))
c_y_test = y_test['nl_seq_cost'].to_numpy().reshape(-1, 1)
rgr = m_regression_model().fit(X_train, c_y_train)
regressors['nl_seq_cost'] = rgr
loss_in_percentage = np.average(np.abs(np.exp(rgr.predict(X_test).reshape(-1,1)) - c_y_test) / c_y_test)
print(f"Loss in percentage: +- {loss_in_percentage * 100}%")

Loss in percentage: +- 7.967411028915744%


In [13]:
# for name, importance in zip(features, rgr.feature_importances_):
#     print(name, "=", importance)

# import matplotlib.pyplot as plt

# importances = rgr.feature_importances_
# indices = np.argsort(importances)

# plt.title('Feature importances for hash join + index scan')
# plt.barh(range(len(indices)), importances[indices], color='b', align='center')
# plt.yticks(range(len(indices)), [features[i] for i in indices])
# plt.xlabel('Relative Importance')
# plt.show()

### merge join + index scan

In [14]:
c_y_train = np.log(y_train['mj_idx_cost'].to_numpy().reshape(-1, 1))
c_y_test = y_test['mj_idx_cost'].to_numpy().reshape(-1, 1)
rgr = m_regression_model().fit(X_train, c_y_train)
regressors['mj_idx_cost'] = rgr

loss_in_percentage = np.average(np.abs(np.exp(rgr.predict(X_test).reshape(-1,1)) - c_y_test) / c_y_test)
print(f"Loss in percentage: +- {loss_in_percentage * 100}%")

Loss in percentage: +- 16.843268118325952%


In [15]:
# for name, importance in zip(features, rgr.feature_importances_):
#     print(name, "=", importance)

# import matplotlib.pyplot as plt

# importances = rgr.feature_importances_
# indices = np.argsort(importances)

# plt.title('Feature importances for hash join + index scan')
# plt.barh(range(len(indices)), importances[indices], color='b', align='center')
# plt.yticks(range(len(indices)), [features[i] for i in indices])
# plt.xlabel('Relative Importance')
# plt.show()
# print(rgr.predict(X_test)[-4])
# print(c_y_test.iloc[-4])

### merge join + seq scan

In [16]:
c_y_train = np.log(y_train['mj_seq_cost'].to_numpy().reshape(-1, 1))
c_y_test = y_test['mj_seq_cost'].to_numpy().reshape(-1, 1)

rgr = m_regression_model().fit(X_train, c_y_train)
regressors['mj_seq_cost'] = rgr

loss_in_percentage = np.average(np.abs(np.exp(rgr.predict(X_test).reshape(-1,1)) - c_y_test) / c_y_test)
print(f"Loss in percentage: +- {loss_in_percentage * 100}%")

Loss in percentage: +- 10.594037839601958%


In [17]:
# for name, importance in zip(features, rgr.feature_importances_):
#     print(name, "=", importance)

# import matplotlib.pyplot as plt

# importances = rgr.feature_importances_
# indices = np.argsort(importances)

# plt.title('Feature importances for hash join + index scan')
# plt.barh(range(len(indices)), importances[indices], color='b', align='center')
# plt.yticks(range(len(indices)), [features[i] for i in indices])
# plt.xlabel('Relative Importance')
# plt.show()

# Use Regression model to do classification

In [18]:
predict_test = []
operators = ['hj_idx_cost', 'hj_seq_cost', 'nl_idx_cost', 'nl_seq_cost','mj_idx_cost', 'mj_seq_cost']

for op_idx, op in enumerate(operators) :
    predict_test.append(regressors[op].predict(X_test))
    
results = np.stack(predict_test, axis=1)
acc = np.sum(np.argmin(results, axis=1).reshape(-1,1).flatten() == y_test['optimal_decision'].to_numpy().flatten()) / len(y_test)
print("Test accuracy using all learned costs: %.2f%%" % (acc*100))

Test accuracy using all learned costs: 82.55%
